In [1]:
from langchain.chat_models import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OllamaEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOllama(
    model="mistral:latest",
    temperature=0.1,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

loader = CSVLoader("./file/data.csv")
docs = loader.load_and_split()

embeddings = OllamaEmbeddings(
    model="mistral:latest"
)

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorStore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorStore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a data analyst. Answer questions using only the given context. Answer in Korean. Don't give me english answer. Don't try to translate the name of the column used in the data. Explain briefly about what the data is about first. And then explain about the data overall.  If you don't know the answer, just say you don't know. Do NOT make it up. And also, do not talk about anything else:\n{context}"),
    ("human", "{question}")
])

chain = {
    "context": retriever,
    "question": RunnablePassthrough()
    } | prompt | llm

chain.invoke("Tell me about the data. Answer in Korean.")

 이 데이터는 CSV 파일 './file/data.csv'에서 추출한 것입니다. 각 행에는 "InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country" 열이 있습니다. 이 데이터는 Facturierung (영수증) 관련 정보입니다. 각 행은 하나의 거래에 대한 정보를 담고 있습니다.

AIMessage(content=' 이 데이터는 CSV 파일 \'./file/data.csv\'에서 추출한 것입니다. 각 행에는 "InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country" 열이 있습니다. 이 데이터는 Facturierung (영수증) 관련 정보입니다. 각 행은 하나의 거래에 대한 정보를 담고 있습니다.')